In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [19]:
header_names = ['#errors',
                'scan#', 
                'motor-target_1', 
                'motor-target_2', 
                'motor-actual_1',
                'motor-actual_2',
                'data_channel_0',
                'data_channel_1', 
                'data_channel_2',
                'data_channel_3',
                'data_channel_4',
                'data_channel_5',
                'data_channel_6',
                'data_channel_7']

data = pd.read_csv('../data/trial_output05.tsv', delimiter='\t', names = header_names)

In [18]:
# Check that all is copacetic
data

,#errors,scan#,motor-target_1,motor-target_2,motor-actual_1,motor-actual_2,data_channel_0,data_channel_1,data_channel_2,data_channel_3,data_channel_4,data_channel_5,data_channel_6,data_channel_7
0,0.0,0.0,9.395,33.448,9.3950,33.448,0.013195,0.010942,-0.006436,-0.019309,-0.031860,-0.005470,1.671540,1.827301
1,0.0,0.0,9.236,33.400,9.2360,33.400,0.003862,0.014804,0.000322,-0.010620,-0.024458,-0.008689,1.370637,1.558580
2,0.0,0.0,9.242,33.433,9.2420,33.433,0.000644,-0.035400,-0.016412,-0.009332,0.010299,0.009333,1.125409,1.304020
3,0.0,0.0,9.380,33.373,9.3800,33.373,-0.004827,-0.036043,-0.007401,-0.010620,0.028321,0.005793,0.923627,1.082929
4,0.0,0.0,9.302,33.457,9.3021,33.457,0.001288,-0.008367,-0.008689,-0.010941,-0.025423,-0.010298,0.755637,0.894985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45623,0.0,11.0,9.290,33.502,9.2900,33.502,-0.001609,-0.032503,-0.016091,-0.014481,0.001931,0.007081,-0.018987,0.005793
45624,0.0,11.0,9.302,33.511,9.3021,33.511,-0.013516,-0.020274,-0.005149,-0.005149,0.004184,-0.006436,-0.016734,0.008368
45625,0.0,11.0,9.275,33.466,9.2750,33.466,-0.017700,-0.037009,-0.009654,0.003219,0.020275,-0.003540,-0.016412,0.008046
45626,0.0,11.0,9.362,33.448,9.3620,33.448,-0.019309,-0.037974,-0.009654,0.004828,0.022528,-0.003540,-0.015769,0.009977


In [15]:
# Notice Scan '11.0' has 1,004 fewer samples
# All points in a scan are guaranteed to be sampled...except scan 11?
# Scans are averaged together, homodyne OKE results
data['scan#'].value_counts().sort_index()

0.0     3886
1.0     3886
2.0     3886
3.0     3886
4.0     3886
5.0     3886
6.0     3886
7.0     3886
8.0     3886
9.0     3886
10.0    3886
11.0    2882
Name: scan#, dtype: int64

In [21]:
# No errors identified in 45,628 measurements!
# An error may consist of a communication error b/w motor & acquisition computer
# Motors will be reinitialized, and the scan is restarted
# 
data['#errors'].value_counts()

0.0    45628
Name: #errors, dtype: int64

In [38]:
# There are 67-motor_1 positions each with b/w 670(9.245)-689(9.236) samples taken
# Motor_1 positions range (9.215, 9.413)
data['motor-target_1'].value_counts().sort_index()

9.215    682
9.218    681
9.221    679
9.224    684
9.227    685
        ... 
9.401    680
9.404    683
9.407    681
9.410    677
9.413    680
Name: motor-target_1, Length: 67, dtype: int64

In [40]:
# step-size for motor_1
# Why is the range of positions greater for motor_1 than motor_2?
(9.413-9.215)/67

0.002955223880597021

In [37]:
# Each motor_2 position has b/w 775(33.481)-793(33.391, 33.400, 33.478) samples taken
# There are 58 motor_2 positions Motor_2 positions range (33.340, 33.511)
data['motor-target_2'].value_counts().sort_index()

33.340    790
33.343    786
33.346    784
33.349    786
33.352    787
33.355    783
33.358    789
33.361    788
33.364    788
33.367    788
33.370    787
33.373    785
33.376    787
33.379    787
33.382    784
33.385    779
33.388    784
33.391    793
33.394    782
33.397    792
33.400    793
33.403    787
33.406    785
33.409    786
33.412    782
33.415    782
33.418    782
33.421    786
33.424    788
33.427    787
33.430    788
33.433    779
33.436    789
33.439    792
33.442    789
33.445    786
33.448    785
33.451    789
33.454    786
33.457    791
33.460    786
33.463    789
33.466    784
33.469    791
33.472    788
33.475    787
33.478    793
33.481    775
33.484    789
33.487    784
33.490    784
33.493    787
33.496    786
33.499    787
33.502    788
33.505    788
33.508    792
33.511    789
Name: motor-target_2, dtype: int64

In [41]:
# step-size for motor_2
(33.511-33.340)/58

0.0029482758620689546

In [ ]:
data = data.drop(columns=['#errors', 'scan#'])
data = data.groupby(['motor-target_1', 'motor-target_2']).mean()
display(data.head(60))
data_to_plot = data['data_channel_0'].reset_index().pivot(columns='motor-target_1', index='motor-target_2')
display(data_to_plot)
sns.heatmap(data_to_plot)

In [30]:
a = np.arange(33.340, 33.511, 58.)
a

array([33.34])

In [28]:
a

[33.34]